In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
import torchvision.models as models

from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import numpy as np
import os
#from google.colab import drive
#from torchsummary import summary


/net/dali/home/mscbio/fea54/.conda/envs/10701-env-hw4/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/net/dali/home/mscbio/fea54/.conda/envs/10701-env-hw4/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [20]:
#!wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-160.tgz
#!tar -xvzf /content/drive/MyDrive/imagenette2.tgz -C /content/drive/MyDrive/

#drive.mount('/content/drive')

In [2]:
path = "~/CPCB/imagenette2"

In [3]:
# Create common transformations for both training and validation
common_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(*imagenet_stats)
])

# Apply transformations directly to datasets
train_dataset = ImageFolder(root=os.path.join(path, 'train/'), transform=common_transforms)
val_dataset = ImageFolder(root=os.path.join(path, 'val/'), transform=common_transforms)

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [4]:
model_names = dir(models)
#print(model_names)

In [5]:
# Load pre-trained AlexNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = models.alexnet(weights='AlexNet_Weights.DEFAULT')

#Change the number of output classes
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.to(device)
model


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

### AlexNet With No Replacement

In [10]:
# Training and evaluation functions
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()  # Set model to evaluation mode
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradients for faster inference
        for inputs, labels in tqdm(dataloader):
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device

            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs, 1)  # Get predictions

            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = correct_predictions / total_samples
    return accuracy

In [34]:
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_loss = np.zeros(num_epochs)
val_accuracy = np.zeros(num_epochs)
for epoch in range(num_epochs):
    train_loss[epoch] = train(model, train_loader, criterion, optimizer, device)
    val_accuracy[epoch] = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss[epoch]:.4f}, Val Accuracy: {val_accuracy[epoch]:.4f}")

100%|██████████| 62/62 [00:10<00:00,  5.98it/s]


Epoch 1, Train Loss: 2.2886, Val Accuracy: 0.1804


100%|██████████| 62/62 [00:10<00:00,  5.95it/s]


Epoch 2, Train Loss: 2.0251, Val Accuracy: 0.3824


100%|██████████| 62/62 [00:10<00:00,  5.97it/s]


...

Epoch 8, Train Loss: 0.5860, Val Accuracy: 0.7478


100%|██████████| 62/62 [00:10<00:00,  6.02it/s]


Epoch 9, Train Loss: 0.5115, Val Accuracy: 0.7462


100%|██████████| 62/62 [00:10<00:00,  6.02it/s]

Epoch 10, Train Loss: 0.4770, Val Accuracy: 0.7730


In [35]:
print(train_loss)
print(val_accuracy)

[2.28862987 2.02510441 1.54558145 1.09885341 0.9047092  0.78913322
 0.66759123 0.58603918 0.51151736 0.47700033]
[0.18038217 0.38242038 0.59133758 0.68535032 0.71923567 0.76127389
 0.71210191 0.7477707  0.74624204 0.77299363]


## AlexNet With Layer Replaced

In [19]:
model_replaced = model
model_replaced.to(device)

# Freeze all parameters first
for param in model_replaced.parameters():
    param.requires_grad = False

new_layer = nn.Sequential(
     nn.Linear(13, 13)
)

# Replace the 10th layer with the new Linear layer
model_replaced.features[10] = new_layer

for param in model_replaced.classifier[6].parameters():
    param.requires_grad = True

for param in model_replaced.parameters():
    param.requires_grad = True


optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_replaced.parameters()), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [20]:
model_replaced.to(device)
model_replaced

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Sequential(
      (0): Linear(in_features=13, out_features=13, bias=True)
    )
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, 

In [21]:
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_replaced.parameters(), lr=0.001)
train_loss_replaced = np.zeros(num_epochs)
val_accuracy_replaced = np.zeros(num_epochs)
for epoch in range(num_epochs):
    train_loss_replaced[epoch] = train(model_replaced, train_loader, criterion, optimizer, device)
    val_accuracy_replaced[epoch] = evaluate(model_replaced, val_loader, device)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss_replaced[epoch]:.4f}, Val Accuracy: {val_accuracy_replaced[epoch]:.4f}")

100%|██████████| 62/62 [00:12<00:00,  5.14it/s]


Epoch 1, Train Loss: 1.4907, Val Accuracy: 0.6678


100%|██████████| 62/62 [00:10<00:00,  5.91it/s]


Epoch 2, Train Loss: 0.8725, Val Accuracy: 0.7465


100%|██████████| 62/62 [00:10<00:00,  5.96it/s]


...

Epoch 8, Train Loss: 0.3162, Val Accuracy: 0.8023


100%|██████████| 62/62 [00:10<00:00,  5.90it/s]


Epoch 9, Train Loss: 0.2797, Val Accuracy: 0.7941


100%|██████████| 62/62 [00:10<00:00,  5.86it/s]

Epoch 10, Train Loss: 0.2739, Val Accuracy: 0.8013


In [22]:
print(train_loss_replaced)
print(val_accuracy_replaced)

[1.49068893 0.87254947 0.64506043 0.52592281 0.43934388 0.37632574
 0.32683861 0.31624952 0.2796693  0.27392561]
[0.6677707  0.74649682 0.79414013 0.79031847 0.78471338 0.81605096
 0.80585987 0.80229299 0.79414013 0.80127389]


In [ ]:
plt.plot(epoch, train_loss_changed, label="Training Loss with Layer Removed")
plt.plot(epoch, train_loss, label="Baseline Training Loss")
plt.title("Training Loss of ResNet18 on Imagenette")
plt.ylabel("Training Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()



In [ ]:
#val_accuracies = [0.97503185 0.96968153 0.97299363 0.97019108 0.9722293  0.97375796 0.97171975 0.96968153 0.97248408 0.97146497]

epoch = [i for i in range(1,num_epochs+1)]
plt.plot(epoch, val_accuracy, label="Validation Accuracy with Layer Removed")
plt.plot(epoch, val_accuracies, label="Baseline Validation Accuracy")
plt.title("Validation Accuracy of AlexNet on Imagenette")
plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Epoch")

plt.show()
print(val_accuracy)